In [161]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from catboost import CatBoostClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import time
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix



# Same features

We will train a CatBoost model on the Titanic dataset.

In [75]:
titanic = sns.load_dataset('titanic')
titanic['age'].fillna(titanic['age'].median(), inplace=True)
titanic.drop(columns=['who', 'embarked', 'parch', 'fare', 'deck', 'embark_town', 'alive',"alone"], inplace=True)

In [77]:
X = titanic.drop(columns=['survived'])
y = titanic['survived']

cat_features = ["sex", "class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [78]:
# Initialize CatBoost Classifier
catboost_model = CatBoostClassifier(iterations=1000, 
                                    learning_rate=0.05, 
                                    depth=6, 
                                    verbose=100)

# Train the model
catboost_model.fit(X_train, y_train, 
                   eval_set = (X_test, y_test),
                   cat_features=cat_features,plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6580557	test: 0.6605953	best: 0.6605953 (0)	total: 3.15ms	remaining: 3.14s
100:	learn: 0.3623712	test: 0.4389407	best: 0.4342884 (54)	total: 149ms	remaining: 1.33s
200:	learn: 0.3290721	test: 0.4464352	best: 0.4342884 (54)	total: 304ms	remaining: 1.21s
300:	learn: 0.2948500	test: 0.4622076	best: 0.4342884 (54)	total: 461ms	remaining: 1.07s
400:	learn: 0.2705016	test: 0.4815637	best: 0.4342884 (54)	total: 625ms	remaining: 934ms
500:	learn: 0.2543097	test: 0.4819783	best: 0.4342884 (54)	total: 810ms	remaining: 806ms
600:	learn: 0.2365760	test: 0.5020827	best: 0.4342884 (54)	total: 999ms	remaining: 663ms
700:	learn: 0.2223127	test: 0.5150031	best: 0.4342884 (54)	total: 1.2s	remaining: 511ms
800:	learn: 0.2089500	test: 0.5183471	best: 0.4342884 (54)	total: 1.39s	remaining: 344ms
900:	learn: 0.1998820	test: 0.5337496	best: 0.4342884 (54)	total: 1.56s	remaining: 172ms
999:	learn: 0.1909176	test: 0.5398363	best: 0.4342884 (54)	total: 1.76s	remaining: 0us

bestTest = 0.4342883903
b

In [79]:
# Make predictions
y_pred_cb = catboost_model.predict(X_test)

# Evaluate the Cat_boost
cb_accuracy = accuracy_score(y_test, y_pred_cb)
print(f"CatBoost Accuracy: {cb_accuracy:.4f}")
print(classification_report(y_test, y_pred_cb))

CatBoost Accuracy: 0.8212
              precision    recall  f1-score   support

           0       0.80      0.92      0.86       105
           1       0.86      0.68      0.76        74

    accuracy                           0.82       179
   macro avg       0.83      0.80      0.81       179
weighted avg       0.83      0.82      0.82       179



# Other features

Try adding other categorical features. Does your accuracy improve?

In [142]:
titanic = sns.load_dataset('titanic')
titanic['age'].fillna(titanic['age'].median(), inplace=True)
# Manipulate the columns that you add

# Regression

### Now, try doing regression instead of classification. 
Using the California housing dataset, train regression models using Decision Trees, Random Forest, and CatBoost. Then, plot them using sns.regplot. If the points are annoying, set scatter = False. What are their R2?

In [146]:
# Import necessary libraries
from catboost import CatBoostRegressor
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd

# Load the California housing dataset
data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

